In [48]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from xgboost import XGBClassifier


In [49]:
pd.set_option('display.max_columns', None)

train_data = pd.read_csv('/Users/janbanot/Dev/uni/msc-cs-code/sem2/JSWAD/kaggle/playground-series-s3e22/train.csv')
test_data = pd.read_csv('/Users/janbanot/Dev/uni/msc-cs-code/sem2/JSWAD/kaggle/playground-series-s3e22/test.csv')
sample_data = pd.read_csv('/Users/janbanot/Dev/uni/msc-cs-code/sem2/JSWAD/kaggle/playground-series-s3e22/sample_submission.csv')

horses = train_data.drop(['id', 'hospital_number', 'lesion_2', 'lesion_3', 'outcome'], axis=1).astype({
    'lesion_1': 'object'
})
y = train_data[['outcome']]
horses.info()
horses

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                1235 non-null   object 
 1   age                    1235 non-null   object 
 2   rectal_temp            1235 non-null   float64
 3   pulse                  1235 non-null   float64
 4   respiratory_rate       1235 non-null   float64
 5   temp_of_extremities    1196 non-null   object 
 6   peripheral_pulse       1175 non-null   object 
 7   mucous_membrane        1214 non-null   object 
 8   capillary_refill_time  1229 non-null   object 
 9   pain                   1191 non-null   object 
 10  peristalsis            1215 non-null   object 
 11  abdominal_distention   1212 non-null   object 
 12  nasogastric_tube       1155 non-null   object 
 13  nasogastric_reflux     1214 non-null   object 
 14  nasogastric_reflux_ph  1235 non-null   float64
 15  rect

,surgery,age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,cp_data
0,yes,adult,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,no
1,yes,adult,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,no
2,yes,adult,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,no
3,yes,adult,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,yes
4,no,adult,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,yes,adult,38.5,129.0,48.0,cool,reduced,pale_pink,more_3_sec,depressed,absent,moderate,none,more_1_liter,2.0,NaN,distend_large,57.0,66.0,serosanguious,2.0,yes,2206,no
1231,yes,adult,37.5,60.0,50.0,cool,reduced,pale_cyanotic,less_3_sec,mild_pain,hypomotile,slight,slight,none,3.0,decreased,distend_small,35.0,6.4,serosanguious,3.6,yes,2209,yes
1232,yes,young,37.5,84.0,40.0,normal,reduced,normal_pink,less_3_sec,mild_pain,hypomotile,slight,slight,none,3.0,increased,firm,40.0,5.9,cloudy,7.0,yes,400,yes
1233,yes,adult,38.1,70.0,16.0,normal,reduced,bright_red,less_3_sec,mild_pain,hypomotile,slight,none,more_1_liter,2.0,absent,distend_small,58.0,74.0,cloudy,2.0,yes,2209,no


In [50]:
# Count missing values and print column type if missing values are present
missing_counts = horses.isnull().sum()
for col, count in missing_counts.items():
    if count > 0:
        print(f"Column '{col}' has {count} missing values. Type: {horses[col].dtype}")

for col in horses.select_dtypes('object').columns:
    horses[col] = horses[col].fillna('Unknown')
horses = horses.infer_objects(copy=False)

Column 'temp_of_extremities' has 39 missing values. Type: object
Column 'peripheral_pulse' has 60 missing values. Type: object
Column 'mucous_membrane' has 21 missing values. Type: object
Column 'capillary_refill_time' has 6 missing values. Type: object
Column 'pain' has 44 missing values. Type: object
Column 'peristalsis' has 20 missing values. Type: object
Column 'abdominal_distention' has 23 missing values. Type: object
Column 'nasogastric_tube' has 80 missing values. Type: object
Column 'nasogastric_reflux' has 21 missing values. Type: object
Column 'rectal_exam_feces' has 190 missing values. Type: object
Column 'abdomen' has 213 missing values. Type: object
Column 'abdomo_appearance' has 48 missing values. Type: object


/var/folders/pb/wvnr47ns7t38d_w08j9rswzm0000gn/T/ipykernel_65085/2764344049.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  horses[col] = horses[col].fillna('Unknown')


In [51]:
# Encode categorical columns and assign to horses_encoded
horses_encoded = horses.copy()
for col in horses_encoded.select_dtypes('object').columns:
    horses_encoded[col] = LabelEncoder().fit_transform(horses_encoded[col].astype(str))

y_encoded = LabelEncoder().fit_transform(y.values.ravel())

In [52]:
horses_encoded

,surgery,age,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,cp_data
0,1,0,38.1,132.0,24.0,2,4,3,3,2,1,4,3,1,6.5,2,2,57.0,8.5,3,3.4,1,2209,0
1,1,0,37.5,88.0,12.0,2,3,5,3,4,1,1,1,2,2.0,1,2,33.0,64.0,3,2.0,1,2208,0
2,1,0,38.3,120.0,28.0,2,4,6,2,3,4,1,3,3,3.5,0,1,37.0,6.4,3,3.4,1,5124,0
3,1,0,37.1,72.0,30.0,1,4,6,3,4,4,1,3,2,2.0,2,2,53.0,7.0,2,3.9,1,2208,1
4,0,0,38.0,52.0,48.0,3,3,4,2,1,4,2,3,1,7.0,4,4,47.0,7.3,2,2.6,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1,0,38.5,129.0,48.0,2,4,6,3,2,1,1,1,2,2.0,0,1,57.0,66.0,3,2.0,1,2206,0
1231,1,0,37.5,60.0,50.0,2,4,5,2,4,4,4,3,3,3.0,2,2,35.0,6.4,3,3.6,1,2209,1
1232,1,1,37.5,84.0,40.0,3,4,4,2,4,4,4,3,3,3.0,3,3,40.0,5.9,2,7.0,1,400,1
1233,1,0,38.1,70.0,16.0,3,4,2,2,4,4,4,1,2,2.0,1,2,58.0,74.0,2,2.0,1,2209,0


In [53]:
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(horses_encoded, y)

/Users/janbanot/Dev/uni/msc-cs-code/.venv/lib/python3.12/site-packages/sklearn/base.py:1363: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [54]:
xgb_classifier = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_classifier.fit(horses_encoded, y_encoded)

/Users/janbanot/Dev/uni/msc-cs-code/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [11:20:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'


In [55]:
# Prepare and encode test data
test_encoded = test_data.drop(['id', 'hospital_number', 'lesion_2', 'lesion_3'], axis=1).astype({
    'lesion_1': 'object'
})

# Fill nulls in object columns with 'Unknown'
for col in test_encoded.select_dtypes('object').columns:
    test_encoded[col] = test_encoded[col].fillna('Unknown')
test_encoded = test_encoded.infer_objects(copy=False)

# Encode categorical columns using the same approach as for training data
for col in test_encoded.select_dtypes('object').columns:
    test_encoded[col] = LabelEncoder().fit_transform(test_encoded[col].astype(str))

/var/folders/pb/wvnr47ns7t38d_w08j9rswzm0000gn/T/ipykernel_65085/4043055718.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_encoded[col] = test_encoded[col].fillna('Unknown')


In [56]:
# test_predictions = rf_classifier.predict(test_encoded)
# train_predictions = rf_classifier.predict(horses_encoded)

test_predictions = xgb_classifier.predict(test_encoded)
train_predictions = xgb_classifier.predict(horses_encoded)


train_f1_score = f1_score(y_encoded, train_predictions, average='micro')
print("Training F1 Score:", train_f1_score)

Training F1 Score: 1.0


In [57]:
# After predicting on test set
test_predictions = xgb_classifier.predict(test_encoded)
test_predictions_labels = LabelEncoder().fit(y.values.ravel()).inverse_transform(test_predictions)

submission_df = pd.DataFrame({'id': test_data['id'], 'outcome': test_predictions_labels})
submission_df.to_csv('submission.csv', index=False)